# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [32]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [33]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,toliara,-23.3500,43.6667,20.06,70,0,6.20,MG,2023-11-11
1,1,puerto ayora,-0.7393,-90.3518,24.19,92,32,2.68,EC,2023-11-11
2,2,sosnovka,56.2550,51.2842,2.28,91,100,5.01,RU,2023-11-11
3,3,ilulissat,69.2167,-51.1000,-6.99,68,75,6.69,GL,2023-11-11
4,4,borogontsy,62.6683,131.1683,-24.86,92,100,1.06,RU,2023-11-11


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [57]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    frame_width = 800,
    frame_height = 500,
    scale = 1,
    alpha = .5,
    tiles = "OSM",
    size = "Humidity",
    color = "City"
)

# Display the map
humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [39]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_temp_df = city_data_df[(city_data_df["Max Temp"]>=18) & (city_data_df["Max Temp"]<24)]
ideal_clouds_df = ideal_temp_df[ideal_temp_df["Cloudiness"]<59]
ideal_humidity_df = ideal_clouds_df[(ideal_clouds_df["Humidity"] > 25) &(ideal_clouds_df["Humidity"] < 80)]

# Drop any rows with null values
ideal_humidity_df = ideal_humidity_df.dropna(how='any').reset_index()

# Display sample data
# ideal_humidity_df.info()
ideal_humidity_df.head()

,index,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,0,toliara,-23.3500,43.6667,20.06,70,0,6.20,MG,2023-11-11
1,13,13,carnarvon,-24.8667,113.6333,23.65,55,1,8.29,AU,2023-11-11
2,26,26,blackmans bay,-43.0167,147.3167,18.18,58,42,2.68,AU,2023-11-11
3,52,52,papatowai,-46.5619,169.4708,18.40,43,0,2.10,NZ,2023-11-11
4,58,58,invercargill,-46.4000,168.3500,19.45,52,2,4.47,NZ,2023-11-11


### Step 3: Create a new DataFrame called `hotel_df`.

In [45]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_humidity_df[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = pd.Series(dtype=object)

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,toliara,MG,-23.3500,43.6667,70,NaN
1,carnarvon,AU,-24.8667,113.6333,55,NaN
2,blackmans bay,AU,-43.0167,147.3167,58,NaN
3,papatowai,NZ,-46.5619,169.4708,43,NaN
4,invercargill,NZ,-46.4000,168.3500,52,NaN


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [50]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "limit":1,
    "filter":f"",
    "bias":f"",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{row['Lng']},{row['Lat']},{radius}"
    params["bias"] = f"proximity:{row['Lng']},{row['Lat']}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
toliara - nearest hotel: Ambary
carnarvon - nearest hotel: No hotel found
blackmans bay - nearest hotel: Villa Howden
papatowai - nearest hotel: No hotel found
invercargill - nearest hotel: The Grand
uberlandia - nearest hotel: Hotel Presidente
sitia - nearest hotel: Itanos Hotel
tura - nearest hotel: No hotel found
manta - nearest hotel: La Gaviota
nova sintra - nearest hotel: Residência Ka Dencho
i-n-salah - nearest hotel: باجودة
murzuq - nearest hotel: فندق باريس - مرزق
tsiombe - nearest hotel: No hotel found
sangaria - nearest hotel: No hotel found
yazman - nearest hotel: Yazman Banquet Marriage Hall
ambovombe - nearest hotel: La source
sirte - nearest hotel: فندق المدينة-سرت
lahan - nearest hotel: Shrawan ke Room
al jawf - nearest hotel: No hotel found
timimoun - nearest hotel: أناف
awjilah - nearest hotel: No hotel found
tolanaro - nearest hotel: Hôtel Mahavokey
taltal - nearest hotel: Hostería Taltal
whakatane - nearest hotel: Whakatane Hotel
praia - neares

,City,Country,Lat,Lng,Humidity,Hotel Name
0,toliara,MG,-23.3500,43.6667,70,Ambary
1,carnarvon,AU,-24.8667,113.6333,55,No hotel found
2,blackmans bay,AU,-43.0167,147.3167,58,Villa Howden
3,papatowai,NZ,-46.5619,169.4708,43,No hotel found
4,invercargill,NZ,-46.4000,168.3500,52,The Grand
5,uberlandia,BR,-18.9186,-48.2772,26,Hotel Presidente
6,sitia,GR,35.2000,26.0986,79,Itanos Hotel
7,tura,IN,25.5198,90.2201,67,No hotel found
8,manta,EC,-0.9500,-80.7333,78,La Gaviota
9,nova sintra,CV,14.8667,-24.7167,69,Residência Ka Dencho


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [59]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    frame_width = 800,
    frame_height = 500,
    scale = 1,
    alpha = .5,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    hover_cols = "Hotel Name"
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name)